In [1]:
import pandas as pd
import numpy as np
import re
import time 

import requests as rq
import bs4 as bs4
import json


# Coleta de dados da barra de busca  

In [3]:
queries = ["financas",'ficcao-cientifica','exatas']

# Coletandos os dados das páginas direcionados pela barra de busca do site 
for query in queries:
    for i in range(1,21):
        if i == 1:
            url =(f'https://livros.mercadolivre.com.br/livros/{query}')  
        else:
            page = (48*i - 47)
            url =(f'https://livros.mercadolivre.com.br/livros/{query}/_Desde_{page}')

        print(url)
        response = rq.get(url)
        with open(f'{query}_pg.{i}.html', 'w+',encoding='utf-8') as output:
            output.write(response.text)
            time.sleep(2)            

https://livros.mercadolivre.com.br/livros/financas
https://livros.mercadolivre.com.br/livros/financas/_Desde_49
https://livros.mercadolivre.com.br/livros/financas/_Desde_97
https://livros.mercadolivre.com.br/livros/financas/_Desde_145
https://livros.mercadolivre.com.br/livros/financas/_Desde_193
https://livros.mercadolivre.com.br/livros/financas/_Desde_241
https://livros.mercadolivre.com.br/livros/financas/_Desde_289
https://livros.mercadolivre.com.br/livros/financas/_Desde_337
https://livros.mercadolivre.com.br/livros/financas/_Desde_385
https://livros.mercadolivre.com.br/livros/financas/_Desde_433
https://livros.mercadolivre.com.br/livros/financas/_Desde_481
https://livros.mercadolivre.com.br/livros/financas/_Desde_529
https://livros.mercadolivre.com.br/livros/financas/_Desde_577
https://livros.mercadolivre.com.br/livros/financas/_Desde_625
https://livros.mercadolivre.com.br/livros/financas/_Desde_673
https://livros.mercadolivre.com.br/livros/financas/_Desde_721
https://livros.mercad

In [4]:
# Extraindo os link para os anuncios 
for query in queries:
    for i in range(1,21):
            
        with open(f'{query}_pg.{i}.html','r+',encoding='utf-8') as inp:
            page_html = inp.read()
            
            parsed = bs4.BeautifulSoup(page_html)
            
            tags = parsed.findAll('a')
            
            for e in tags:
                
                if e.has_attr('class') and e.has_attr('title'):
            
                    if e['class'] == ['ui-search-link']:
                   
                        link = e['href']
                        title = e['title']
               
                
                        with open("parsed_videos.json", 'a+') as output:
                            data = {"link": link, "title": title,"query" : query}
                            output.write("{}\n".format(json.dumps(data)))

In [ ]:
df = pd.read_json("parsed_videos.json", lines=True)

# Coleta de dados dos Anuncios 

In [7]:
# Acessando cada link e convertendo para um formato HTML
lista_links = df['link'].unique()

for link in lista_links:
    response = rq.get(link)
   
    try:
        with open(f'{link[36:49]}.html', 'w+',encoding='utf-8') as output:
            output.write(response.text)
            time.sleep(2)
    except:
        continue

In [9]:
# Extraindo os dados brutos
price = []
sales = []
titles = []
author = []
language = []
editora = []
formato = []
subgen = []




for link in lista_links:
    
    try:
        with open(f'{link[36:49]}.html','r+',encoding='utf-8') as inp:
                    page_html = inp.read()
            
                    parsed = bs4.BeautifulSoup(page_html)
            
                    # Coletando os dados de Preço 
                    class_price = parsed.find_all(attrs={"itemprop":re.compile(r"price")})[0]
                    price.append(float(class_price['content']))
            
           
                    # Coletando total de vendas
                    try:
                        class_sales = parsed.find_all(attrs={"class":re.compile(r"ui-pdp-subtitle")})
                        sales.append(class_sales[0].text.strip())
                    except:
                        continue    
                
           
        
        
                # Coletando os titulos
                    try:
                        class_title = parsed.find_all(attrs={"class":re.compile(r"andes-table__column--value")})[0]
                        titles.append( class_title.text.strip())
                    except:
                        continue
        
                # Coletando os dados do autor
             
                    try:
                        class_author = parsed.find_all(attrs={"class":re.compile(r"andes-table__column--value")})[1]
                        author.append(class_author.text.strip())
                    except:
                        continue
                    
               # Coletando os dados de idioma
                    try:
                        class_language = parsed.find_all(attrs={"class":re.compile(r"andes-table__column--value")})[2]
                        language.append(class_language.text.strip())
                    except:
                        continue
            
               # Coletando os dados da editora
                    try:
                        class_editora = parsed.find_all(attrs={"class":re.compile(r"andes-table__column--value")})[3]
                        editora.append(class_editora.text.strip())
                    except:
                        continue    
            
               # Coletando os dodos de formato
                    try:
                        class_format = parsed.find_all(attrs={"class":re.compile(r"andes-table__column--value")})[5]
                        formato.append(class_format.text.strip())
                    except:
                        continue
                    
               # Coletando os dados de subgenero 
                    try:
                        class_subgen = parsed.find_all(attrs={"class":re.compile(r"andes-breadcrumb__link")})[2]
                        subgen.append(class_subgen.text.strip())
                    except:
                        continue
    except:
        continue

In [10]:
# Convertendo os dados brutos para um DATAFRAME
titles = pd.DataFrame(titles,columns=['titles'])
price = pd.DataFrame(price,columns=['price'])
sales = pd.DataFrame(sales,columns=['sales'])
author = pd.DataFrame(author,columns=['author'])
formato = pd.DataFrame(formato,columns=['formato'])
editora = pd.DataFrame(editora,columns=['editora'])
language = pd.DataFrame(language,columns=['language'])



df = pd.concat([titles,price,sales,author,formato,editora,language],axis=1)
df.to_feather('raw_data_unclean.feather')


In [15]:
ndf = pd.read_feather('raw_data_unclean.feather')